In [8]:
with open(r"C:\Users\selas\Downloads\taylorswift.txt", 'r') as f:
    text=f.read()

text
tokens=text.encode("utf-8")
tokens=list(tokens)

In [9]:
class BasicTokenizer():
     

    def get_stats(self, ids):
            
            
            
            counts={}
            for pair in zip(ids, ids[1:]):
                counts[pair]=counts.get(pair, 0)+1
            return counts


    def merge(self, ids, pair, idx):
        
        new_ids=[]
        i=0
        while i<len(ids):
            if i < len(ids) - 1 and ids[i]==pair[0] and ids[i+1]==pair[1]:
                new_ids.append(idx)
                i+=2
            else:
                new_ids.append(ids[i])
                i+=1
            #print(f"merging{pair}----->{idx}")
        return new_ids

    
    

        
    def train(self, text, vocab_size):
        tokens=text.encode("utf-8")
        tokens=list(tokens)
        
 
            
        
        num_merges=vocab_size-256
        ids=tokens
        merges={}
        vocab={idx: bytes([idx]) for idx in range(256)}
        idx=256
        for i in range(num_merges):
            stats=self.get_stats(ids)
            #print(stats)
            pair=max(stats, key=stats.get)
            ids=self.merge(ids, pair, idx)
            merges[pair]=idx
            '''if i%1000==0:
                print(f"merging({pair}) to {idx})")'''
            #print(len(ids))
            vocab[idx] = vocab[pair[0]] + vocab[pair[1]]
            idx+=1
        
        self.merges=merges
        self.vocab=vocab
        #return(merges)
        print(f"compression ratio: {len(tokens) / len(ids):.2f}X")

         
        

    def encode(self, text):
        tokens=text.encode("utf-8")
        tokens=list(tokens)
        while len(tokens)>=2:
            stats=self.get_stats(tokens)
            #print(self.merges)
            pair=min(stats, key=lambda p: self.merges.get(p, float('inf')))
            if pair not in self.merges:
                break
            idx=self.merges[pair]
            tokens=self.merge(tokens, pair, idx)
        return tokens

        
    def decode(self, ids):
        tokens=b"".join(self.vocab[idx] for idx in ids)
        text=tokens.decode("utf-8", errors='replace')
        return text
            



In [10]:

tokin=BasicTokenizer()
tokin.train(text, 500)


compression ratio: 2.32X


In [18]:
print(tokin.decode(tokin.encode("ordinary bro")))

ordinary bro
